In [1]:
# Execute if necessary
# %%capture
# !pip install numpy seaborn matplotlib pandas openml

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, Tuple, Union, List
import openml
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score


# Práctica 4: Overfitting

__Instrucciones__: A continuación hay una lista de funciones que debe implementar o tareas que debe desarrollar. La descripción de cada una de ellas se encuentra en la definición de cada una de las funciones. Cada función está marcada por &#x1F625;,  &#x1F643; o &#x1F921;. Las marcas indican:

- &#x1F625;: Indican una entrega que debe ser hecha dentro de la misma sesión de la asignación. 
- &#x1F643;: Indican una entrega que puede ser hecha hasta la siguiente sesión.
- &#x1F921;: Debe mostrar un avance en la misma sesión, pero la entrega puede ser hecha en la siguiente.

Aquellas entregas parciales que no sean hechas el día de la asignación ya no serán válidas para las entregas totales, sin embargo, las entregas totales seguirán siendo válidas.

En esta sección se incluye un dataset real. El dataset importado se llama LDPA y puede leer su descripción en la siguiente liga

https://www.openml.org/d/1483

In [ ]:
# Descarga la metadata del dataset
dataset_info = openml.datasets.get_dataset(1483, download_data=False)

# Obtiene el nombre de la columna a predecir
target = dataset_info.default_target_attribute

(
    features, # Dataframe con las características que se pueden utilizar para predecir
    outputs, # Columna a predecir
    categorical_mask, # Máscara que indica que columnas de todas las características son categoricas
    columns # Lista con el nombre de las características
)= dataset_info.get_data(
    dataset_format="dataframe", target=target
)

categorical_mask = np.array(categorical_mask)
columns = np.array(columns)

In [ ]:
print(f"La columna a predecir se llama '{target}'")
print(f"Todas las características son {str(columns)}")
print(f"Las características categóricas son {str(columns[categorical_mask])}")
print(f"Las características numéricas son {str(columns[~categorical_mask])}")

In [ ]:
# Impresión de las características
features

In [ ]:
outputs

A continuación se realiza la partición de __train__ y __test__. __No debe utilizar la partición de test por ningún motivo__.

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    features, 
    outputs, 
    test_size=0.5, 
    random_state=11
)

## Asignación 1 &#x1F921;

Utilizando las técnicas vistas en clase, realicé su mejor esfuerzo para obtener el mejor modelo lineal posible utilizando __únicamente el conjunto de entrenamiento__ (X_train y y_train). Puede utilizar bibliotecas, pero únicamente las técnicas vistas en clase y debe ir generando métricas o visualizaciones que respalden su toma de decisiones. Debe tener e imprimir una estimación de $E_{out}$ utilizando su conjunto de entrenamiento. La métrica utilizada será la entropía cruzada.

##### Data set Information:

People used for recording of the data were of the data were wearing four tags (ankle left, angle right, belt and chest). Each instance is a localization data for one of the tags. The tag can be identified by one of the attributes.

##### Attribute Information:
Instance Example: 
A01,020-000-033-111,633790226057226795,27.05.2009 14:03:25:723,4.292500972747803,2.0738532543182373,1.36650812625885,walking

1. Sequence Name {A01,A02,A03,A04,A05,B01,B02,B03,B04,B05,C01,C02,C03,C04,C05,D01,D02,D03,D04,D05,E01,E02,E03,E04,E05} (Nominal)
    - A, B, C, D, E = 5 people

2. Tag identificator {010-000-024-033,020-000-033-111,020-000-032-221,010-000-030-096} (Nominal)
    - ANKLE_LEFT = 010-000-024-033
    - ANKLE_RIGHT = 010-000-030-096
    - CHEST = 020-000-033-111
    - BELT = 020-000-032-221

3. timestamp (Numeric) all unique
4. date FORMAT = dd.MM.yyyy HH:mm:ss:SSS (Date)
5. x coordinate of the tag (Numeric)
6. y coordinate of the tag (Numeric)
7. z coordinate of the tag (Numeric)
8. activity {walking,falling,'lying down',lying,'sitting down',sitting,'standing up from lying','on all fours','sitting on the ground','standing up from sitting','standing up from sitting on the ground'} (Nominal)

# Preprocesamiento

In [ ]:
fig1 = plt.figure("Filtro")
fig1.subplots_adjust(hspace=0.5, wspace=0.5)

for i in range(1, 5):
    # p, q = filtro(p, q)

    ax = fig1.add_subplot(2, 2, i)
    # ax.plot(p,q,"g--")
    sns.histplot(x=X_train.iloc[:, i-1], kde=True, line_kws={'linestyle' : 'dashed',
                                                    'linewidth' : '2'}, color='blue').lines[0].set_color('red')
    ax.set_xlabel("valores")
    ax.set_ylabel("frecuencia")
    ax.set_title("{}".format(X_train.columns[i-1]))
    ax.grid(color='gray', linestyle='dashed', linewidth=1, alpha=0.4)
    # Pintar los ejes pasando por (0,0)
    ax.axhline(0, color='black', linewidth=0.5)

# Creamos otra figura, se mostrar
fig2 = plt.figure("n ** i")
fig2.subplots_adjust(hspace=0.5, wspace=0.5)

# x = list(range(1, 10))
for i in range(4, 7):
    # y = npotencia(x, i)
    ax = fig2.add_subplot(2, 2, i-3)
    # ax.plot(x, y, "r-.")
    sns.histplot(x=X_train.iloc[:, i], kde=True, line_kws={'linestyle' : 'dashed',
                                                    'linewidth' : '2'}, color='blue').lines[0].set_color('red')
    
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title("{}".format(X_train.columns[i]))
    ax.grid(color='gray', linestyle='dashed', linewidth=1, alpha=0.4)
    ax.axhline(0, color='black', linewidth=0.5)

In [ ]:
# Comprobamos las clases de la variable a predecir
plt.figure()
sns.histplot(x=y_train, kde=True, line_kws={'linestyle' : 'dashed',
                                                    'linewidth' : '2'}, color='orange').lines[0].set_color('red')
plt.title(f'Datos de y_train')
plt.show()

plt.figure()
sns.histplot(x=outputs, kde=True, line_kws={'linestyle' : 'dashed',
                                                    'linewidth' : '2'}, color='green',).lines[0].set_color('red')
plt.title(f'Datos originales, antes de separar')
plt.show()

Como se puede ver las clases estan mal distribuidos, por lo que no sería bueno seguir.

Continuando con el preprosesamiento de los datos, aplicamos un one hot encode a los datos de entrenamiento para cambiar los valores categoricos a numericos.

In [ ]:
# Creamos una instancía para realizar el one hot encode
transformer = make_column_transformer(
    (sklearn.preprocessing.OneHotEncoder(), ['V1', 'V2']),
    remainder='passthrough'
)

# Creamos un dataFrame con los datos ya convertidos a one hot 
x_train_one_hot = pd.DataFrame(transformer.fit_transform(X_train), columns=transformer.get_feature_names_out())

Aplicamos una regresión logistica multinomial

In [ ]:
# Creamos una instancia de pipeline
pipe = Pipeline([('scaler', sklearn.preprocessing.StandardScaler()), 
                 ('logistic', LogisticRegression(class_weight=None, max_iter=1000, random_state=42, multi_class='multinomial'))])

# Entrenamos el modelo
pipe.fit(x_train_one_hot, y_train)

# Predecimos la probabilidad de que ocurra un valor
valor_predicho = pipe.predict_proba(x_train_one_hot)

# Obtenemos validación cruzada
score = cross_val_score(pipe, x_train_one_hot, y_train, cv=5)
print(f'El score de validación cruzada, es: {score}')

# Obtenemos el valor de accuracy
print(f'Con un accuracy score de: {sklearn.metrics.accuracy_score(y_train, pipe.predict(x_train_one_hot))}')

# Obtenemos el valor de la entropia cruzada
print(f'El valor de entropia cruzada es: {sklearn.metrics.log_loss(y_test, valor_predicho, labels=pipe.classes_)}')

## Asignación 2 &#x1F921;

Evalue su modelo final en el conjunto de test (X_test y y_test). Su práctica será evaluada acorde a las técnicas aplicadas, la estimación de $E_{out}$, el valor de $E_{test}$, y contra las métricas obtenidas por sus compañeros.

In [ ]:
# Creamos una instancía para realizar el one hot encode
transformer = make_column_transformer(
    (sklearn.preprocessing.OneHotEncoder(), ['V1', 'V2']),
    remainder='passthrough'
)

# Creamos un dataFrame con los datos ya convertidos a one hot 
x_test_one_hot = pd.DataFrame(transformer.fit_transform(X_test), columns=transformer.get_feature_names_out())
# x_test_one_hot.head(5)

In [ ]:
# Creamos otra instancia de pipeline
# pip = Pipeline([('scaler', sklearn.preprocessing.StandardScaler()), 
#                  ('logistic', LogisticRegression(class_weight=None, max_iter=1000, random_state=42, multi_class='multinomial'))])

pipe.fit(x_test_one_hot, y_test)

# Predecimos la probabilidad de que ocurra un valor
valor_predicho_test = pipe.predict_proba(x_test_one_hot)

# Obtenemos validación cruzada
score = cross_val_score(pipe, x_test_one_hot, y_test, cv=5)
print(f'El score de validación cruzada, es: {score}')

# Obtenemos el valor de accuracy
print(f'Con un accuracy score de: {sklearn.metrics.accuracy_score(y_train, pipe.predict(x_test_one_hot))}')

# Obtenemos el valor de la entropia cruzada
print(f'El valor de entropia cruzada es: {sklearn.metrics.log_loss(y_test, valor_predicho_test, labels=pipe.classes_)}')